# TMS awake/asleep comparison- Consciousness

## Load in data

This has to be converted to a fif file first using the bv2fif.py script

In [ ]:
from bv2fif import bv2fif
import os,re,glob
import os.path as op
from MEEGbuddy import MEEGbuddy
import numpy as np
from mne import read_epochs
from mne.io import Raw
import matplotlib.pyplot as plt

subject = 'Test'
session = '01'

stimuli = {'TMS':['Response/R 15',-0.5,1.0]}
baseline = ['Response/R 15',-0.4,0]

datarex = re.compile(r'[%s|%s]*.vhdr' %(subject.lower(),subject.upper()))
correx = re.compile(r'[%s|%s]*.bvct' %(subject.lower(),subject.upper()))
    
if True:
    files = glob.glob(op.join(os.getcwd(),'data','TMS',subject,session,'*'))
    if not files:
        print('Subject data must be in a folder with their name')
    corf = None
    for f in files:
        if correx.search(f):
            corf = f
    for f in files:
        if datarex.search(f) and not 'avg' in f and not 'TMS' in op.basename(f):
            fdata = (os.path.join(os.path.dirname(f),
                     op.basename(f).split('.')[0]+'-raw.fif'))
            epof = (os.path.join(os.path.dirname(f),
                    op.basename(f).split('.')[0]+'-epo.fif'))
            if not op.isfile(fdata) or not op.isfile(epof):
                bv2fif(f,corf,ref_ch='Fp1')
            #
            fs_dir = op.join(os.getcwd(),'data','TMS')
            bemf = op.join(fs_dir,subject,'bem','%s-5120-5120-5120-bem-sol.fif' %(subject))
            srcf = op.join(fs_dir,subject,'bem','%s-oct-6-src.fif' %(subject))
            transf = op.join(fs_dir,subject,'COR-%s_%s.fif' %(subject,session))
            #
            epochs = read_epochs(epof)
            print('Loading in to MEEGbuddy %s' %(subject))
            data = MEEGbuddy(subject=subject,session=session,fdata=fdata,baseline=baseline,stimuli=stimuli,task='TMS-EEG',eeg=True,
                             subjects_dir=op.join(os.getcwd(),'data'),exclude_response=[0,len(epochs)-1],
                             epochs=epochs,event='TMS',fs_subjects_dir=fs_dir,bemf=bemf,srcf=srcf,transf=transf)

Extracting parameters from ./JM/1_31_2018_JM_con_pilot.vhdr...
Setting channel info structure...
Reading 0 ... 99771499  =      0.000 ...  3990.860 secs...


### Mark Bads

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    print(data.subject + ' ' + data.session)
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_raw(keyword='Preprocessed'):
        data.filterRaw(keyword_out='Preprocessed',l_freq=0.5,overwrite=True)
        data.makeEpochs(keyword_in='Preprocessed',keyword_out='Vis',overwrite=True)
        data.filterEpochs(event,keyword_in='Vis',h_freq=40,l_freq=None,overwrite=True) 
        data.plotEvoked(event,keyword='Vis',show=True,detrend=1,downsample=False,ylim={'eeg':[-100,100]})
        data.plotRaw(keyword_in='Preprocessed',l_freq=None,h_freq=None,overwrite=True)

### Choose and Apply Interpolation Raw data

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    print(data.subject + ' ' + data.session)
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_raw(keyword='Interpolated'):
        raw_interp,_ = data.plotInterpolateArtifact('TMS',keyword='Preprocessed',mode='spline',npoint_art=11,offset=1,tmin=-0.1,tmax=0.1,ylim=[-1e-4,1e-4])
        plt.show()
        data.applyInterpolation(raw_interp,keyword_out='Interpolated',overwrite=True)

### Do ICA on Raw Data

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_ICA(keyword='ICA'):
        # precompute ICA
        data.findICA(keyword_in='Interpolated',keyword_out='ICA',overwrite=True,overwrite_ica=True)
        #
        ok = True
        while ok:
            data.plotICA(keyword_in='Interpolated',keyword_out='ICA')
            ok = input('Press Enter to continue...')
        #
        data.makeEpochs(keyword_in='ICA',keyword_out='Vis',overwrite=True)
        data.filterEpochs(event,keyword_in='Vis',h_freq=40,overwrite=True)
        data.plotEvoked(event,keyword='Vis',ylim={'eeg':[-60,40]},tmin=-0.5,tmax=0.5)
        plt.show()

### Multitaper Spectrogram and Sleep/Wake Assignment

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_PSD_image(keyword='ICA',ch='Oz'): #FIX THIS
        state_times = data.psdMultitaper(keyword='ICA',overwrite=False,ch='Oz',vmax=-10)
        data.assignConditionFromStateTimes('TMS',state_times)

## Epochs

### Plot Epochs

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_epochs(event,keyword='Clean'):
        data.plotEvoked(event,show=True,detrend=1,downsample=False,ylim={'eeg':[-100,100]})
        data.plotEpochs(event,keyword_out='Clean',l_freq=None,h_freq=None,overwrite=True)

### Interpolate Artifact

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_epochs(event,keyword='Interpolated'):
        epo_interp,_ = data.plotInterpolateArtifact(event,use_raw=False,keyword='Clean',mode='spline',
                                                    npoint_art=11,offset=1,tmin=-0.1,tmax=0.1,ylim=[-6e-5,4e-5])
        data.applyInterpolation(epo_interp,event,keyword_out='Interpolated',overwrite=True)
        plt.show()

### Apply ICA to Epochs

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_epochs(event,keyword='ICA'):
        data.findICA(event=event,keyword_in='Interpolated',keyword_out='ICA')

### QC ICA

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not data._has_epochs(event,keyword='Clean'):
        # Quality check ICA (will need to do multiple times if the components are hand-selected) (click on the component name to see topo)
        ok = True
        while ok:
            try:
                data.plotICA(event=event,keyword_in='ICA')
                data.filterEpochs(event,h_freq=40,keyword_in='ICA',keyword_out='Vis',overwrite=True)
                data.plotEvoked(event,show=True,detrend=1,keyword='Vis',ylim={'eeg':[-100,100]})
                data.plotTopo(event,condition='State',keyword='Vis',
                              show=True,detrend=1,downsample=False)
                data.plotEpochs(event,keyword_in='ICA',keyword_out='Clean',overwrite=True)
                ok = input('Press ok to continue, type anything to go back')
                while ok:
                    data.filterEpochs(event,h_freq=40,keyword_in='Clean',keyword_out='Vis',overwrite=True)
                    data.plotEvoked(event,show=True,detrend=1,keyword='Vis',ylim={'eeg':[-100,100]})
                    data.plotTopo(event,condition='State',keyword='Vis',
                                  show=True,detrend=1,downsample=False)
                    data.plotEpochs(event,keyword_in='Clean',overwrite=True)
            except Exception as e:
                print(e)
                print(data.subject)
            ok = input('Press enter to move on, type anything to go back\n')

## Source Space

In [1]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not all([data._has_source(event,'State',value) for value in data._default_values('State')]):
        epochs = data._load_epochs(event,keyword='Clean')
        data._save_epochs(epochs.crop(tmin=-0.5,tmax=0),'Baseline',keyword='Clean')
        data.epochs2source(event,'State',keyword_in='Clean')

NameError: name 'os' is not defined

### Plot Source Space (optional)

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    data.plotSourceSpace(event,'State',keyword='ICA',tmin=-0.1,tmax=0.5)

## PCI

The interesting part!

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    if not all([data._has_noreun_PCI(event,'State',value,keyword='Clean') 
                for value in data._default_values('State')]):
        data.noreunPhi('TMS','State',keyword_in='Clean',npoint_art=10,
                       tmin=0.0,tmax=0.3,shared_baseline=False,
                       bl_tmin=-0.4,bl_tmax=-0.01,downsample=False,
                       recalculate_PCI=True)

In [ ]:
from MEEGbuddy import loadMEEGbuddies
import os.path as op
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

data_struct = loadMEEGbuddies(op.join(os.getcwd(),'data'),task='TMS-EEG')

for data in tqdm(data_struct):
    event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
    data.plotNoreunPCI('TMS','State',keyword='Clean')